
Nombre del Trabajo: AI for Good: Human Heritage Automatic pre-Hispanic to Spanish Recogniser and Translator

Nombre del asesor del proyecto: Dr. Juan Arturo Nolazco Flores

Equipo: 20 Nombre de alumnos:

* Juan Carlos Garza Sanchez A00821522
* Sinaí Avalos Rivera A01730466

In [ ]:
import pandas as pd

# Construcción

Cargamos el dataset y cambiamos el nombre de las columnas a spanish y nahuatl

In [ ]:
df = pd.read_csv("hf://datasets/somosnlp-hackathon-2022/Axolotl-Spanish-Nahuatl/train.csv")
df = df[['sp', 'nah']].rename(columns={'sp': 'spanish', 'nah': 'nahuatl'})
df

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,spanish,nahuatl
0,"Y así, cuando hizo su ofrenda de fuego, se sie...",Auh in ye yuhqui in on tlenamacac niman ye ic ...
1,"¿Si es jade, si es oro, acaso no tendrá que ir...","¿In chalchihuitl, teocuitlatl, mach ah ca on yaz?"
2,"Y cuando el Sol estuvo solo en el cielo, enseg...",Auh yn oyuh in yoca hualmotlalli tonatiuh ylhu...
3,Allá acudieron asimismo los señores del cabild...,Yn oncan mohuicatza yhuan yn ciudad cabildo tl...
4,Usos:,Kualtia:
...,...,...
20023,"El Espíritu y la esposa dicen: ""¡Ven!""El que ...",On Espíritu Santo niman isihuau on Borreguito ...
20024,Yo advierto a todo el que oye las palabras de...,Nemechtlachicahuilia nenmochimej yejhuan nenqu...
20025,y si alguno quita de las palabras del libro d...,Niman tla yacaj quipopolohuilis itemachtil Dio...
20026,"El que da testimonio de estas cosas dice: ""¡S...","On yejhuan quipantlantia yejhua in, ijquin qui..."


Imprimimos la forma del dataset

In [ ]:
print("Shape of dataset", df.shape)

Shape of dataset (20028, 2)


Limpiamos el dataset eliminando filas donde no tengamos la traducción en español o en nahuatl

In [ ]:
print(df.isnull().sum())

missing_data_rows = df[df.isnull().any(axis=1)]

print(missing_data_rows)

spanish    4
nahuatl    4
dtype: int64
     spanish nahuatl
2444     NaN     NaN
4524     NaN     NaN
5399     NaN     NaN
8400     NaN     NaN


In [ ]:
df = df.dropna(axis=0, how='any')

In [ ]:
print(df.isnull().sum())

spanish    0
nahuatl    0
dtype: int64


Cambiamos el tipo de dato a string para poder tratarlo como cadenas de texto

In [ ]:
df = df.astype(str)

Debido a las características del dataset, y aún más, del lenguaje prehispánico, la mejor opción es seleccionar trabajar bajo la estrategia "Few Shots Learning" (FSL) con Gemini. Puesto que FSL permite que un modelo preentenado aprenda patrones con solo unos cuantos ejemplos, sin necesidad de un dataset masivo. Se necesitan ejemplos específicos de la lengua prehispánica para ajustarse a su gramática y estructura. Además de que, Gemini no requiere que previamente hagamos tokenización puesto que ya lo maneja internamente así como el procesamiento del texto, por lo que no es necesario tokenizar el dataset.

# Normalización


Cambiamos todas las palabras a minusculas ya que en ambos lenguajes la capitalización no representa algún cambio de significado.

In [ ]:
df['spanish'] = df['spanish'].str.lower()
df['nahuatl'] = df['nahuatl'].str.lower()

In [ ]:
df

,spanish,nahuatl
0,"y así, cuando hizo su ofrenda de fuego, se sie...",auh in ye yuhqui in on tlenamacac niman ye ic ...
1,"¿si es jade, si es oro, acaso no tendrá que ir...","¿in chalchihuitl, teocuitlatl, mach ah ca on yaz?"
2,"y cuando el sol estuvo solo en el cielo, enseg...",auh yn oyuh in yoca hualmotlalli tonatiuh ylhu...
3,allá acudieron asimismo los señores del cabild...,yn oncan mohuicatza yhuan yn ciudad cabildo tl...
4,usos:,kualtia:
...,...,...
20023,"el espíritu y la esposa dicen: ""¡ven!""el que ...",on espíritu santo niman isihuau on borreguito ...
20024,yo advierto a todo el que oye las palabras de...,nemechtlachicahuilia nenmochimej yejhuan nenqu...
20025,y si alguno quita de las palabras del libro d...,niman tla yacaj quipopolohuilis itemachtil dio...
20026,"el que da testimonio de estas cosas dice: ""¡s...","on yejhuan quipantlantia yejhua in, ijquin qui..."


# Seleccion / extracción

Vamos a utilizar las oraciones para hacer few shot learning con Gemini. No es necesario hacer extracción o selección de features.

# Conclusion general de la fase de Preparación de los datos en el contexto de la metodología CRISP-ML

En esta fase, se preprocesó el dataset paa garantizar su calidad y compatibilidad, obteniendo:

* Caracteristicas del dataset: El dataset cuenta con 20024 oraciones en nahuatl y 20024 oraciones en español, lo que proporciona una base suficiente para el entrenamiento del modelo.
* Eliminamos 4 filas del dataset que no contenían información y así evitar inconsistencias.
* Cambiamos el tipo de dato del dataset a string, asegurando que el modelo trabaje correctamente con texto.
* Cambiamos todas las oraciones a minusculas para reducir la variabilidad en las representaciones lingüisticas.
* Debido a que los transformers pre entrenados que utilizaremos no requieren tokenización previa, no realizaremos tokenización, ya que manejan este proceso internamente.
* Impacto del preprocesamiento:
  * Limpieza de datos: Garantiza que el modelo trabaje con datos relevantes sin ruido.
  * Consistencia del formato: Facilita el entrenamiento al evitar problemas con mayúsculas y tipos de datos.
  * Eficiencia computacional: Eliminar filas vacías y evitar procesar datos innecesarios.